## Remove backgrounds

In [ ]:
import os

dataset_root = '/content/drive/MyDrive/ball_detection/datasets/ball'
splits = ['train', 'valid', 'test']
image_extensions = ['.jpg', '.png', '.jpeg']

for split in splits:
    images_dir = os.path.join(dataset_root, split, 'images')
    labels_dir = os.path.join(dataset_root, split, 'labels')

    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        continue

    for img_fname in os.listdir(images_dir):
        img_name, img_ext = os.path.splitext(img_fname)
        if img_ext.lower() not in image_extensions:
            continue

        label_fname = img_name + '.txt'
        label_path = os.path.join(labels_dir, label_fname)
        img_path = os.path.join(images_dir, img_fname)

        # Remove if label missing
        if not os.path.exists(label_path):
            print(f"Removing image with missing label: {img_path}")
            os.remove(img_path)
            continue

        # Remove if label empty
        with open(label_path, 'r') as f:
            content = f.read().strip()
            if content == '':
                print(f"Removing empty label and image: {label_path}")
                os.remove(label_path)
                os.remove(img_path)


Removing empty label and image: /content/drive/MyDrive/ball_detection/datasets/ball/train/labels/4b770a_1_2_png.rf.cc814a4adb61ad0478f3c20ef9bb6de1.txt
Removing empty label and image: /content/drive/MyDrive/ball_detection/datasets/ball/train/labels/798b45_7_3_png_jpg.rf.85b4ba8c87777c190c0128b73374b82e.txt
Removing empty label and image: /content/drive/MyDrive/ball_detection/datasets/ball/train/labels/744b27_3_3_png_jpg.rf.c2181df15731068938e642d42ce4c4e7.txt
Removing empty label and image: /content/drive/MyDrive/ball_detection/datasets/ball/train/labels/798b45_5_5_png_jpg.rf.c4ef871c62dfef249f01746717567fb0.txt
Removing empty label and image: /content/drive/MyDrive/ball_detection/datasets/ball/train/labels/08fd33_0_3_png.rf.d34586ebf014e4a9571db487918d45bc.txt
Removing empty label and image: /content/drive/MyDrive/ball_detection/datasets/ball/train/labels/798b45_5_1_png_jpg.rf.72ab3fdf343794faa806d66961bc7f2b.txt
Removing empty label and image: /content/drive/MyDrive/ball_detection/da

## Copy dataset

In [2]:
import shutil
import os

source_folder = '/content/drive/MyDrive/ball_detection/datasets/ball'
destination_folder = '/content/drive/MyDrive/ball_detection/datasets/ball_copy'

# Create the destination folder if it doesn't exist
# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# Copy the entire folder
shutil.copytree(source_folder, destination_folder)
print(f"Folder copied from '{source_folder}' to '{destination_folder}'")

Folder copied from '/content/drive/MyDrive/ball_detection/datasets/ball' to '/content/drive/MyDrive/ball_detection/datasets/ball_copy'


In [ ]:
!cp -r /content/drive/MyDrive/ball_detection/datasets/ball /content/drive/MyDrive/ball_detection/datasets/ball_copy

In [1]:
!rm -rf /content/drive/MyDrive/ball_detection/datasets/ball_copy

## Copy-Paste Augmentation

In [ ]:
import os
import cv2
import random

def copy_paste_augmentation(dataset_root):

    image_dir = os.path.join(dataset_root, "train/images")
    label_dir = os.path.join(dataset_root, "train/labels")

    image_files = sorted([
        f for f in os.listdir(image_dir)
        if f.lower().endswith((".jpg", ".png", ".jpeg"))
    ])

    for img_name in image_files:

        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_dir, img_name.rsplit(".", 1)[0] + ".txt")

        if not os.path.exists(label_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue

        h, w = img.shape[:2]

        # load + FIX malformed labels
        raw = [ln.strip() for ln in open(label_path) if ln.strip()]
        fixed = []
        for ln in raw:
            p = ln.split()
            for i in range(0, len(p), 5):
                chunk = p[i:i+5]
                if len(chunk) == 5:
                    fixed.append(" ".join(chunk))

        if len(fixed) == 0:
            continue

        # first ball
        cls, xc, yc, bw, bh = map(float, fixed[0].split())

        # pixel coords
        x1 = int((xc - bw / 2) * w)
        y1 = int((yc - bh / 2) * h)
        x2 = int((xc + bw / 2) * w)
        y2 = int((yc + bh / 2) * h)

        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            continue

        crop_h, crop_w = crop.shape[:2]

        # random anywhere
        paste_x = random.randint(0, w - crop_w)
        paste_y = random.randint(0, h - crop_h)

        img[paste_y:paste_y+crop_h, paste_x:paste_x+crop_w] = crop

        new_xc = (paste_x + crop_w/2) / w
        new_yc = (paste_y + crop_h/2) / h
        new_bw = crop_w / w
        new_bh = crop_h / h

        fixed.append(f"{int(cls)} {new_xc:.6f} {new_yc:.6f} {new_bw:.6f} {new_bh:.6f}")

        cv2.imwrite(img_path, img)
        open(label_path, "w").write("\n".join(fixed) + "\n")

    print("DONE – copy-paste applied to ALL images.")


In [ ]:
copy_paste_augmentation(
    dataset_root="/content/drive/MyDrive/ball_detection/datasets/ball_copy"
)


DONE – copy-paste applied to ALL images.


## Modular Copy-Paste Logic

In [4]:
import os
import cv2
import random


# ---------------------------
# 1. LOAD IMAGE + LABELS
# ---------------------------
def load_image_and_labels(image_dir, label_dir, img_name):
    img_path = os.path.join(image_dir, img_name)
    label_path = os.path.join(label_dir, img_name.rsplit(".", 1)[0] + ".txt")

    if not os.path.exists(label_path):
        return None, None, None, None

    img = cv2.imread(img_path)
    if img is None:
        return None, None, None, None

    with open(label_path, "r") as f:
        labels = [ln.strip() for ln in f.readlines() if ln.strip()]

    return img, labels, img_path, label_path


# ---------------------------
# 2. GET FIRST BALL CROP
# ---------------------------
def extract_crop(img, label_line):
    h, w = img.shape[:2]

    try:
        cls, xc, yc, bw, bh = map(float, label_line.split())
    except:
        return None, None, None, None, None

    # YOLO → pixel
    x1 = int((xc - bw / 2) * w)
    y1 = int((yc - bh / 2) * h)
    x2 = int((xc + bw / 2) * w)
    y2 = int((yc + bh / 2) * h)

    # clamp coordinates
    x1 = max(0, min(x1, w-1))
    y1 = max(0, min(y1, h-1))
    x2 = max(0, min(x2, w))
    y2 = max(0, min(y2, h))

    crop = img[y1:y2, x1:x2]
    if crop.size == 0:
        return None, None, None, None, None

    return crop, (x1, y1, x2, y2), (cls, xc, yc, bw, bh), w, h


# ---------------------------
# 3. CHOOSE PASTE LOCATION: MIDDLE RECTANGLE ONLY
# ---------------------------
def choose_paste_location_middle(w, h, crop_w, crop_h, jitter_frac=0.15):
    """
    Always place the crop somewhere inside the middle (center) 1/3 x 1/3 cell.
    jitter_frac controls how much random offset inside that cell (0.0 = exact center).
    """
    # center cell bounds (pixel)
    cell_w = w // 3
    cell_h = h // 3
    cell_x0 = cell_w     # start x of center cell
    cell_y0 = cell_h     # start y of center cell

    # center of the cell
    center_x = cell_x0 + cell_w // 2
    center_y = cell_y0 + cell_h // 2

    # jitter range (in pixels)
    max_jitter_x = int(jitter_frac * cell_w)
    max_jitter_y = int(jitter_frac * cell_h)

    jitter_x = random.randint(-max_jitter_x, max_jitter_x)
    jitter_y = random.randint(-max_jitter_y, max_jitter_y)

    paste_x = center_x + jitter_x - crop_w // 2
    paste_y = center_y + jitter_y - crop_h // 2

    # clamp so crop is fully inside image
    paste_x = max(0, min(paste_x, w - crop_w))
    paste_y = max(0, min(paste_y, h - crop_h))

    return paste_x, paste_y


# ---------------------------
# 4. PASTE CROP ON IMAGE
# ---------------------------
def paste_crop(img, crop, paste_x, paste_y):
    crop_h, crop_w = crop.shape[:2]
    img[paste_y:paste_y+crop_h, paste_x:paste_x+crop_w] = crop


# ---------------------------
# 5. ADD NEW LABEL
# ---------------------------
def add_new_label(labels, cls, paste_x, paste_y, crop_w, crop_h, w, h):
    new_xc = (paste_x + crop_w / 2) / w
    new_yc = (paste_y + crop_h / 2) / h
    new_bw = crop_w / w
    new_bh = crop_h / h

    labels.append(
        f"{int(cls)} {new_xc:.6f} {new_yc:.6f} {new_bw:.6f} {new_bh:.6f}"
    )
    return labels


# ---------------------------
# MAIN FUNCTION
# ---------------------------
def copy_paste_augmentation_middle_cell(dataset_root, n=3, jitter_frac=0.15):
    """
    dataset_root: path to dataset_root that contains train/images and train/labels
    n: apply augmentation every n-th image
    jitter_frac: how much random jitter inside the center cell (0..1)
    """
    image_dir = os.path.join(dataset_root, "train/images")
    label_dir = os.path.join(dataset_root, "train/labels")

    image_files = sorted([
        f for f in os.listdir(image_dir)
        if f.lower().endswith((".jpg", ".png", ".jpeg"))
    ])

    count = 0
    for idx, img_name in enumerate(image_files):

        if idx % n != 0:
            continue

        img, labels, img_path, label_path = load_image_and_labels(image_dir, label_dir, img_name)
        if img is None or labels is None:
            continue

        crop, bbox, (cls, xc, yc, bw, bh), w, h = extract_crop(img, labels[0])
        if crop is None:
            continue

        crop_h, crop_w = crop.shape[:2]

        paste_x, paste_y = choose_paste_location_middle(w, h, crop_w, crop_h, jitter_frac=jitter_frac)

        paste_crop(img, crop, paste_x, paste_y)
        labels = add_new_label(labels, cls, paste_x, paste_y, crop_w, crop_h, w, h)

        # Save results
        cv2.imwrite(img_path, img)
        with open(label_path, "w") as f:
            f.write("\n".join(labels) + "\n")

        count += 1

    print(f"Copy-paste augmentation (middle cell) completed. Modified {count} images.")


In [5]:
copy_paste_augmentation_middle_cell(
    dataset_root="/content/drive/MyDrive/ball_detection/datasets/ball_copy",
    n=3,              # every 3rd image
    jitter_frac=0.15  # small random offset inside center rectangle
)

Copy-paste augmentation (middle cell) completed. Modified 203 images.
